### Charger les données

Avant de nettoyer les données, il faut les charger :

In [ ]:
import pandas as pd

# Charger un fichier CSV
df = pd.read_csv("data.csv")
pd.set_option('display.max_columns', None) #permet d'afficher toutes les colonnes

# Afficher les premières lignes
print(df.head())

On peut aussi charger d'autres formats comme Excel ```(pd.read_excel())```, JSON ```(pd.read_json())``` ou SQL ```(pd.read_sql())```.

### Comprendre les données
Avant de nettoyer, il faut examiner l'état des données :

In [ ]:
# Afficher des infos générales
print(df.info())

# Résumé statistique des colonnes numériques
print(df.describe())

# Voir les valeurs uniques d'une colonne
print(df["colonne"].unique())


### Modifier les données

On peut réaliser plusieurs modifications pour plus de lisibilité.

In [ ]:
df.rename(columns={
            'price': 'prix(million)', 
            'area': 'air(m2)',
            'bedrooms' : 'chambres',
            'bathrooms' : 'sdb',
            'stories' : 'étages',
            'mainroad' : 'route principale',
            'guestroom' : 'chambre ami',
            'basement' : 'sous sol',
            'hotwaterheating' : 'chauffage au gaz',
            'airconditioning' : 'climatisation',
            'parking' : 'parking',
            'prefarea' : 'résidentiel',
            'furnishingstatus' : 'meublé',
            }, inplace=True)

### Gérer les valeurs manquantes
Les valeurs manquantes sont un problème courant. Voici comment les détecter et les traiter :  

- Détection des valeurs manquantes :

In [ ]:
print(df.isnull().sum())  # Compter le nombre de NaN par colonne

- Supprimer les lignes ou colonnes avec des valeurs manquantes :

In [ ]:
df = df.dropna()  # Supprime toutes les lignes avec au moins un NaN
df = df.dropna(axis=1)  # Supprime les colonnes avec au moins un NaN

- Remplacer les valeurs manquantes :

In [ ]:
df["colonne"] = df["colonne"].fillna("valeur_par_défaut")  # Remplir avec une valeur spécifique
df["colonne"] = df["colonne"].fillna(df["colonne"].mean())  # Remplir avec la moyenne

### Gérer les doublons
Les doublons peuvent fausser les analyses :

In [ ]:
# Détection des doublons
print(df.duplicated().sum())
# ou aussi
df.loc[df.duplicated()]

# Suppression des doublons
df = df.drop_duplicates()


### Correction des types de données
Parfois, les données sont mal typées :

In [ ]:
# Convertir une colonne en numérique
df["colonne"] = pd.to_numeric(df["colonne"], errors="coerce")

# Convertir une colonne en date
df["date_colonne"] = pd.to_datetime(df["date_colonne"], errors="coerce")
